In [0]:
spark.conf.set(
    "fs.azure.account.key.datalakest21.dfs.core.windows.net",
    "kMt/NysUCwy25kcAmJ4sYGKq9CR68G00e5Wz0sqlMy5URjC1ByAYKN7Ead+2VatYwdjsHneFF1ss+AStiyfI3Q==")


In [0]:
display(dbutils.fs.ls("abfss://raw@datalakest21.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://raw@datalakest21.dfs.core.windows.net/retail_store_inventory.csv,retail_store_inventory.csv,6191463,1733240004000
abfss://raw@datalakest21.dfs.core.windows.net/transactions.csv,transactions.csv,3513,1733154981000


Extract

In [0]:
df = spark.read.csv("abfss://raw@datalakest21.dfs.core.windows.net/transactions.csv", header=True, inferSchema="true");
df.show(25)

+--------------+----------+----------------+------------------+----------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type|
+--------------+----------+----------------+------------------+----------------+
|             1|        45|      2024-01-01|             100.5|         Deposit|
|             2|        12|      2024-01-02|            200.75|      Withdrawal|
|             3|        78|      2024-01-03|             150.0|         Deposit|
|             4|        34|      2024-01-04|            300.25|      Withdrawal|
|             5|        56|      2024-01-05|             250.0|         Deposit|
|             6|        23|      2024-01-06|             175.0|      Withdrawal|
|             7|        89|      2024-01-07|             225.5|         Deposit|
|             8|        67|      2024-01-08|            275.75|      Withdrawal|
|             9|        14|      2024-01-09|             325.0|         Deposit|
|            10|        92| 

Transformation

In [0]:
from pyspark.sql.functions import current_timestamp
df = df.withColumn("ingestion_timestamp",current_timestamp())
df.show(25)

+--------------+----------+----------------+------------------+----------------+--------------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type| ingestion_timestamp|
+--------------+----------+----------------+------------------+----------------+--------------------+
|             1|        45|      2024-01-01|             100.5|         Deposit|2024-12-04 01:04:...|
|             2|        12|      2024-01-02|            200.75|      Withdrawal|2024-12-04 01:04:...|
|             3|        78|      2024-01-03|             150.0|         Deposit|2024-12-04 01:04:...|
|             4|        34|      2024-01-04|            300.25|      Withdrawal|2024-12-04 01:04:...|
|             5|        56|      2024-01-05|             250.0|         Deposit|2024-12-04 01:04:...|
|             6|        23|      2024-01-06|             175.0|      Withdrawal|2024-12-04 01:04:...|
|             7|        89|      2024-01-07|             225.5|         Deposit|20

Day 1 (Full Load)

In [0]:
#df.write.format("delta").mode("overwrite").save("abfss://processed@abclake1we3.dfs.core.windows.net/delta/transactions")

In [0]:
#validate data
df.write.format("delta").mode("overwrite").save("abfss://processed@datalakest21.dfs.core.windows.net/delta/transactions")

Day 2 (Onwards)

In [0]:
from delta.tables import DeltaTable

existing_data = DeltaTable.forPath(spark, "abfss://processed@datalakest21.dfs.core.windows.net/delta/transactions")

existing_data.alias("existing").merge(df.alias("new"),"existing.transaction_id=new.transaction_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [0]:
#spark.sql("""
 #         CREATE SCHEMA IF NOT EXISTS hive_metastore.transactions
   #       """)

#spark.sql("""
#          CREATE TABLE hive_metastore.transactions.stg_transactions
 #         USING DELTA
  #        LOCATION 'abfss://processed@datalakest21.dfs.core.windows.net/delta/transactions'""")

Validate

In [0]:
spark.read.format("delta").load('abfss://processed@datalakest21.dfs.core.windows.net/delta/transactions').show()

+--------------+----------+----------------+------------------+----------------+--------------------+
|transaction_id|account_id|transaction_date|transaction_amount|transaction_type| ingestion_timestamp|
+--------------+----------+----------------+------------------+----------------+--------------------+
|             1|        45|      2024-01-01|             100.5|         Deposit|2024-12-04 01:05:...|
|             2|        12|      2024-01-02|            200.75|      Withdrawal|2024-12-04 01:05:...|
|             3|        78|      2024-01-03|             150.0|         Deposit|2024-12-04 01:05:...|
|             4|        34|      2024-01-04|            300.25|      Withdrawal|2024-12-04 01:05:...|
|             5|        56|      2024-01-05|             250.0|         Deposit|2024-12-04 01:05:...|
|             6|        23|      2024-01-06|             175.0|      Withdrawal|2024-12-04 01:05:...|
|             7|        89|      2024-01-07|             225.5|         Deposit|20

In [0]:
 #spark.sql("""SELECT * FROM hive_metastore.transactions.stg_transactions""").show(20)

In [0]:
# Disable the retention duration check
#park.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
# VACUUM command to remove old data files
#spark.sql("VACUUM delta.`abfss://processed@datalakest21.dfs.core.windows.net/delta/transactions` RETAIN 72 HOURS")